In [1]:
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from collections import Counter
import joblib

# Cargar y Validar los Datos

In [29]:
# Cargar datos procesados
file_path = '../data/raw/creditcard.csv' 
data = pd.read_csv(file_path)

In [30]:
# Inspección inicial
print("Dimensiones del dataset:", data.shape)
print("Primeras filas del dataset:")
display(data.head())

Dimensiones del dataset: (284807, 31)
Primeras filas del dataset:


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [31]:
print("Distribución de la variable objetivo (Class):")
if 'Class' in data.columns:
    print(data['Class'].value_counts(normalize=True))
else:
    print("La columna 'Class' no está presente en los datos.")

Distribución de la variable objetivo (Class):
Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64


# Manejo de Valores Faltantes

In [32]:
# Verificar valores faltantes
print("\nValores faltantes por columna:")
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])


Valores faltantes por columna:
Series([], dtype: int64)


In [33]:
# Porcentaje de valores faltantes
print("\nPorcentaje de valores faltantes por columna:")
missing_percentage = (missing_values / len(data)) * 100
print(missing_percentage[missing_percentage > 0])


Porcentaje de valores faltantes por columna:
Series([], dtype: float64)


In [34]:
if missing_values.sum() > 0:
    data.fillna(data.mean(), inplace=True)
    print("\nSe imputaron valores faltantes con la media.")
else:
    print("No se encontraron valores faltantes.")

No se encontraron valores faltantes.


# Codificación de Variables Categóricas

In [35]:
categorical_cols = data.select_dtypes(include=['object', 'category']).columns
print("\nColumnas categóricas identificadas:")
print(categorical_cols)


Columnas categóricas identificadas:
Index([], dtype='object')


In [36]:
if len(categorical_cols) > 0:
    # Aplicar One-Hot Encoding
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
    print("\nSe aplicó One-Hot Encoding. Dimensiones del dataset después de la codificación:")
    print(data.shape)
else:
    print("No se encontraron columnas categóricas para codificar.")

No se encontraron columnas categóricas para codificar.


# Dividir el Dataset (Train/Test Split)

In [37]:
# Separar características y etiquetas
X = data.drop(columns=['Class'])  
y = data['Class']

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # Stratify para mantener la proporción de clases
)

In [38]:
print("\nTamaño de los conjuntos:")
print(f"Entrenamiento: {X_train.shape}, Prueba: {X_test.shape}")
print("\nDistribución de clases en el conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True))
print("\nDistribución de clases en el conjunto de prueba:")
print(y_test.value_counts(normalize=True))


Tamaño de los conjuntos:
Entrenamiento: (227845, 30), Prueba: (56962, 30)

Distribución de clases en el conjunto de entrenamiento:
Class
0    0.998271
1    0.001729
Name: proportion, dtype: float64

Distribución de clases en el conjunto de prueba:
Class
0    0.99828
1    0.00172
Name: proportion, dtype: float64


# Selección de Características

In [46]:
# Matriz de correlación
correlation_matrix = data.corr()

In [47]:
# Características altamente correlacionadas con la variable objetivo
correlation_target = correlation_matrix['Class'].sort_values(ascending=False)[1:11]
print("\nCaracterísticas más correlacionadas con la variable objetivo:")
print(correlation_target)


Características más correlacionadas con la variable objetivo:
V11       0.154876
V4        0.133447
V2        0.091289
V21       0.040413
V19       0.034783
V20       0.020090
V8        0.019875
V27       0.017580
V28       0.009536
Amount    0.005632
Name: Class, dtype: float64


In [48]:
# Identificar características con baja correlación con la variable objetivo
low_corr_features = correlation_matrix['Class'][correlation_matrix['Class'] < 0.01].index
if len(low_corr_features) > 0:
    print("\nCaracterísticas con baja correlación con la variable objetivo:")
    print(low_corr_features)
    print("Considera eliminar estas columnas si no tienen un valor analítico evidente.")
else:
    print("No se encontraron características con baja correlación con la variable objetivo.")


Características con baja correlación con la variable objetivo:
Index(['Time', 'V1', 'V3', 'V5', 'V6', 'V7', 'V9', 'V10', 'V12', 'V13', 'V14',
       'V15', 'V16', 'V17', 'V18', 'V22', 'V23', 'V24', 'V25', 'V26', 'V28',
       'Amount'],
      dtype='object')
Considera eliminar estas columnas si no tienen un valor analítico evidente.


In [56]:
# Identificar características altamente correlacionadas entre sí
print("\nCaracterísticas redundantes altamente correlacionadas entre sí:")
redundant_features = []
for col in correlation_matrix.columns:
    high_corr = correlation_matrix[col][correlation_matrix[col] > 0.8].index.drop(col)
    if len(high_corr) > 0:
        redundant_features.append((col, list(high_corr)))

if len(redundant_features) > 0:
    for pair in redundant_features:
        print(f"'{pair[0]}' está altamente correlacionada con {pair[1]}")
else:
    print("No se encontraron características redundantes con alta correlación entre sí.")


Características redundantes altamente correlacionadas entre sí:
No se encontraron características redundantes con alta correlación entre sí.


In [58]:
# Preselección de características
columns_to_drop = list(low_corr_features)
X_train = X_train.drop(columns=columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=columns_to_drop, errors='ignore')

print("\nDimensiones del dataset después de eliminar características irrelevantes o redundantes:")
print(X_train.shape, X_test.shape)


Dimensiones del dataset después de eliminar características irrelevantes o redundantes:
(227845, 8) (56962, 8)


# Estandarización/Escalado de Variables

In [59]:
# Seleccionar columnas numéricas
numeric_cols = X_train.select_dtypes(include=np.number).columns

# Instanciar el escalador
scaler = StandardScaler()

In [60]:
# Ajustar el escalador con los datos de entrenamiento y transformar
X_train_scaled = X_train.copy()
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])

In [61]:
# Transformar el conjunto de prueba con el mismo escalador
X_test_scaled = X_test.copy()
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [62]:
print("Primeras filas del conjunto de entrenamiento escalado:")
display(X_train_scaled.head())

Primeras filas del conjunto de entrenamiento escalado:


,V2,V4,V8,V11,V19,V20,V21,V27
265518,-0.456037,-0.467284,0.957057,-0.110815,-0.555366,-0.173112,0.103171,0.191912
180305,-0.029349,0.175133,0.566704,-0.249329,-0.337959,-0.293564,0.052055,0.006397
42664,0.366065,-0.700918,-4.227592,-0.378786,0.004875,1.663010,-3.807987,0.094137
198723,-0.909447,-1.178149,0.008765,0.092155,0.315458,-0.635236,-0.190540,0.044738
82325,-0.613041,-2.440089,-0.095161,-0.245575,-0.102477,-0.355860,-0.331467,-0.080285


In [3]:
# Guardar el escalador
scaler_path = "../artifacts/scaler.pkl"
joblib.dump(scaler, scaler_path)
print(f"Escalador guardado en: {scaler_path}")

Escalador guardado en: ../artifacts/scaler.pkl


# Manejo del Desbalanceo

In [63]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

print("\nDistribución de clases después de aplicar SMOTE:")
print(Counter(y_train_resampled))


Distribución de clases después de aplicar SMOTE:
Counter({0: 227451, 1: 227451})


# Guardar los Datos Procesados

In [64]:
X_train_resampled.to_csv('../data/processed/X_train.csv', index=False)
y_train_resampled.to_csv('../data/processed/y_train.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

print("\nConjuntos procesados guardados correctamente.")


Conjuntos procesados guardados correctamente.
